In [13]:
from transformers import pipeline
import requests
from langdetect import detect
import torch


In [24]:

# Check if GPU is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1

# Load ASR pipeline with Whisper Tiny and set device to GPU if available
# asr_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-tiny", device=device)
asr_pipeline = pipeline("automatic-speech-recognition", model="rowjak/whisper-tiny-minds14-en", device=device)
classification_pipeline = pipeline("audio-classification", model="rowjak/wav2vec2-minds14-audio-classification", device=device)

# Function to download the audio file
def download_audio(url, filename="audio.mp3"):
    response = requests.get(url)
    with open(filename, "wb") as file:
        file.write(response.content)
    return filename

# Audio URL
audio_url = "http://103.149.203.216:9000/inspira/NLP-C/3F311165F76CA70E0765.wav"
audio_file = download_audio(audio_url)

# Transcribe and detect language
transcription = asr_pipeline(audio_file)
transcribed_text = transcription['text']
detected_language = detect(transcribed_text)

print("Original Text:", transcribed_text)
print("Detected Language:", detected_language)

D:\AppPython\IndonesiaAIBootcamp\NLPBootcamp\.venv\lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Original Text: how do i start a joint account
Detected Language: en


In [25]:

# Perform audio classification using the same model
audio_classification_results = classification_pipeline(audio_file)

# Print classification results
print("Audio Classification Results:", audio_classification_results)

Audio Classification Results: [{'score': 0.07797098904848099, 'label': 'card_issues'}, {'score': 0.07661126554012299, 'label': 'direct_debit'}, {'score': 0.07647158205509186, 'label': 'address'}, {'score': 0.07616636902093887, 'label': 'latest_transactions'}, {'score': 0.07464135438203812, 'label': 'joint_account'}]


In [31]:
from datasets import load_dataset
from transformers import pipeline
import pandas as pd

device = 0 if torch.cuda.is_available() else -1
# Load dataset
dataset = load_dataset("PolyAI/minds14", 'en-US', trust_remote_code=True)

# Load classification pipeline
model_name = "rowjak/wav2vec2-minds14-audio-classification-all"
classifier = pipeline("audio-classification", model=model_name, device=device)




Using the latest cached version of the module from C:\Users\Rowjak\.cache\huggingface\modules\datasets_modules\datasets\PolyAI--minds14\7bdf27fccad4efc6da7c00f739d96bfa0d25a7a85f2244755a73c865f70182bc (last modified on Mon Nov  4 13:34:55 2024) since it couldn't be found locally at PolyAI/minds14, or remotely on the Hugging Face Hub.


config.json:   0%|          | 0.00/3.16k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [28]:
dataset_used = dataset['train'].select(range(100))

In [29]:
dataset_used

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 100
})

In [30]:
import os

intent_classes = {
    0: 'abroad',
    1: 'adress',
    2: 'app_error',
    3: 'atm_limit',
    4: 'balance',
    5: 'business_loan',
    6: 'card_issues',
    7: 'cash_deposite',
    8: 'direct_debit',
    9: 'freeze',
    10: 'latest_transactions',
    11: 'joint_account',
    12: 'high_value_payment',
    13: 'pay_bill'
}

results = []

for entry in dataset['train']:
# for entry in dataset_used:
    filename = os.path.basename(entry["path"])  # Get only the filename
    intent_class = intent_classes[entry["intent_class"]]  # Map index to label
    
    # Get the prediction
    prediction = classifier(entry["path"])
    predicted_intent_class = prediction[0]["label"]
    
    # Append results
    results.append({
        "filename": filename,
        "intent_class": intent_class,
        "prediction_intent_class": predicted_intent_class,
        "transcription" : entry["transcription"]
    })
# 
# # Convert results to DataFrame
# results_df = pd.DataFrame(results)
# 
# # Analyze intent class distribution
# intent_class_distribution = results_df["prediction_intent_class"].value_counts()
# 
# # Display the DataFrame and distribution
# print("Prediction Results:")
# print(results_df)
# print("\nIntent Class Distribution:")
# print(intent_class_distribution)

results_df = pd.DataFrame(results)

# Create DataFrame for intent class distribution
intent_class_distribution_df = results_df["prediction_intent_class"].value_counts().reset_index()
intent_class_distribution_df.columns = ["intent_class", "count"]

# Display the DataFrames
results_df

,filename,intent_class,prediction_intent_class,transcription
0,602ba55abb1e6d0fbce92065.wav,joint_account,balance,I would like to set up a joint account with my...
1,602baf24bb1e6d0fbce922a7.wav,joint_account,cash_deposit,Henry County set up a joint account with my wi...
2,602b9f97963e11ccd901cc32.wav,joint_account,balance,hi I'd like to set up a joint account with my ...
3,602bacab5f67b421554f6488.wav,joint_account,balance,how do I start a joint account
4,602b9d4cbb1e6d0fbce91fa4.wav,joint_account,balance,can you help me set up a joint bank account
...,...,...,...,...
558,602bac4d963e11ccd901cda8.wav,business_loan,balance,hi I'm an account holder have been for 18 year...
559,602ba481963e11ccd901cc94.wav,business_loan,balance,I'm about to make my first payment on my loan ...
560,602b98a5963e11ccd901cba2.wav,business_loan,balance,hi there I'm working about a business loan I w...
561,602ba84f5f67b421554f643e.wav,business_loan,balance,hello I would like to take alone
